<br>

# Login

_Script_ para manejar o e-SAJ.

<br>

Michel Metran\
Data: 20.03.2025\
Atualizado em: 20.03.2025


In [ ]:
import requests
import json
import logging
import requests
from bs4 import BeautifulSoup

import os
import time
from abc import ABC
from pathlib import Path
from typing import List, Literal, Optional, Union
from urllib.parse import unquote

from urllib.parse import urlparse
from pathlib import Path
import pandas as pd
from dotenv import load_dotenv

<br>

-----

## Pasta

In [ ]:
data_path = Path('.').absolute().parents[1] / 'data'
data_path

<br>

---

## Credenciais


In [ ]:
load_dotenv()

USERNAME = os.getenv('USERNAME_TJSP')
PASSWORD = os.getenv('PASSWORD_TJSP')

<br>

-----

## URL

In [ ]:
url = 'https://esaj.tjsp.jus.br/sajcas/login?service=https%3A%2F%2Fesaj.tjsp.jus.br%2Fesaj%2Fj_spring_cas_security_check'

In [ ]:
url_parse = urlparse(url=url)
print(url_parse)
url_parse.query

In [ ]:
url = unquote(url)
url

In [ ]:
url_new = 'https://esaj.tjsp.jus.br/sajcas/login?service=https://esaj.tjsp.jus.br/esaj/j_spring_cas_security_check'

<br>

-----

## Get parameters for POST

In [ ]:
response = requests.get(url=url_new, verify=True)
#r.content.decode('utf-8')
response

In [ ]:
soup = BeautifulSoup(response.text, 'html.parser')
soup

In [ ]:
#soup.find_all(name='input')
element_execution = soup.find(name='input', attrs={'name': 'execution'})
element_execution['value']

In [ ]:
element_lt = soup.find(name='input', attrs={'name': 'lt'})
element_lt['value']

<br>

-----

## POST

In [ ]:
# the payload with your login credentials
payload = {
    'username': USERNAME,
    'password': PASSWORD,
    'lt': element_lt['value'],
    'execution': element_execution['value'],
    '_eventId': 'submit',
    'token': '',
    'pbEntrar': 'Entrar',
    'signature': '',
    'certificadoSelecionado': '',
    'certificado': ''
}
#payload

In [ ]:
s = requests.session()
response = s.post(url=url, data=payload)
response

In [ ]:
#s = requests.session()
response = s.get(url='https://esaj.tjsp.jus.br/esaj/cadastro.do')
response

<br>

----

## BS4

In [ ]:
# parse the HTML content using BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')

# Salvar o objeto soup em um arquivo .html
with open(data_path / 'output.html', 'w', encoding='utf-8') as file:
    file.write(str(soup))

soup

In [ ]:
# find the page title
page_title = soup.title
page_title

<br>

----

## Pesquisa 1º Grau

Mas aqui eu preciso ter o código...

In [ ]:
url = 'https://esaj.tjsp.jus.br/cpopg/show.do?processo.codigo=EX0002XQX0000&processo.foro=564&processo.numero=1500621-87.2025.8.26.0537'

# Dá certo sem o parâmetro Foro
url = 'https://esaj.tjsp.jus.br/cpopg/show.do?processo.codigo=EX0002XQX0000&processo.numero=1500621-87.2025.8.26.0537'

print(url)

In [ ]:
r = s.get(url=url)
r.content

E como encontrar o código?

In [ ]:
n_processo = '1500621-87.2025.8.26.0537'
n_processo_parts = n_processo.split('.')
n_processo_parts_processo_ano = f'{n_processo_parts[0]}.{n_processo_parts[1]}'
n_processo_parts[-1]

In [ ]:
url = f'https://esaj.tjsp.jus.br/cpopg/search.do?conversationId=&cbPesquisa=NUMPROC&numeroDigitoAnoUnificado=1500621-87.2025&foroNumeroUnificado=0537&dadosConsulta.valorConsultaNuUnificado=1500621-87.2025.8.26.0537&dadosConsulta.valorConsultaNuUnificado=UNIFICADO&dadosConsulta.valorConsulta=&dadosConsulta.tipoNuProcesso=UNIFICADO'


url = f'https://esaj.tjsp.jus.br/cpopg/search.do?conversationId=&cbPesquisa=NUMPROC&numeroDigitoAnoUnificado={n_processo_parts_processo_ano}&foroNumeroUnificado={n_processo_parts[-1]}&dadosConsulta.valorConsultaNuUnificado={n_processo}&dadosConsulta.valorConsultaNuUnificado=UNIFICADO&dadosConsulta.valorConsulta=&dadosConsulta.tipoNuProcesso=UNIFICADO'
url

In [ ]:
s = requests.session()
response = s.get(url=url, allow_redirects=False)
#response.content
response.status_code

In [ ]:
# parse the HTML content using BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')

# Salvar o objeto soup em um arquivo .html
with open(data_path / 'output.html', 'w', encoding='utf-8') as file:
    file.write(str(soup))

soup

In [ ]:
response.cookies

In [ ]:
response.headers
#response.headers['Location']